In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Step 1: Load Dataset
dataset_path = "UNSW_NB15_training-set.csv"  # Replace with your dataset path
data = pd.read_csv(dataset_path)

# Select Relevant Parameters for Training
selected_columns = [
    "proto", "service", "sbytes", "dbytes", "sttl", "swin", "dwin", "rate", "smean", "dmean", "attack_cat"
]
data = data[selected_columns]

# Step 2: Preprocessing
# Handle Missing Values
data = data.fillna("Unknown")

# Encode Categorical Features
label_encoders = {}
for col in ['proto', 'service']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Encode Attack Categories (target labels)
attack_cat_encoder = LabelEncoder()
data['attack_cat'] = attack_cat_encoder.fit_transform(data['attack_cat'])

# Separate Features and Target
X = data.drop(['attack_cat'], axis=1)
y = data['attack_cat']

# Scale Numerical Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# One-hot encode the target labels for multi-class classification
y_encoded = to_categorical(y)

# Split into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.3, random_state=42)


In [13]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, SimpleRNN, Input, concatenate

# ANN
ann = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y_encoded.shape[1], activation='softmax')  # Output layer for multi-class classification
])

# CNN
cnn_input = Input(shape=(X_train.shape[1], 1))
cnn = Conv1D(64, kernel_size=3, activation='relu')(cnn_input)
cnn = Flatten()(cnn)
cnn = Dense(y_encoded.shape[1], activation='softmax')(cnn)

# LSTM
lstm_input = Input(shape=(X_train.shape[1], 1))
lstm = LSTM(64, return_sequences=False, activation='relu')(lstm_input)
lstm = Dense(y_encoded.shape[1], activation='softmax')(lstm)

# RNN
rnn_input = Input(shape=(X_train.shape[1], 1))
rnn = SimpleRNN(64, return_sequences=False, activation='relu')(rnn_input)
rnn = Dense(y_encoded.shape[1], activation='softmax')(rnn)

# Combine Outputs
combined = concatenate([ann.output, cnn, lstm, rnn])
final_output = Dense(y_encoded.shape[1], activation='softmax')(combined)

# Define the Stacked Model
stacked_model = Model(inputs=[ann.input, cnn_input, lstm_input, rnn_input], outputs=final_output)
stacked_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape Data for CNN, LSTM, and RNN
X_train_reshaped = np.expand_dims(X_train, axis=-1)  # Reshape for CNN, LSTM, RNN inputs
X_test_reshaped = np.expand_dims(X_test, axis=-1)

# Train the Hybrid Model
stacked_model.fit(
    [X_train, X_train_reshaped, X_train_reshaped, X_train_reshaped],
    y_train,
    epochs=10,
    batch_size=64,
    validation_data=([X_test, X_test_reshaped, X_test_reshaped, X_test_reshaped], y_test)
)


Epoch 1/10
901/901 [==============================] - 19s 16ms/step - loss: 1.0472 - accuracy: 0.6733 - val_loss: 0.8435 - val_accuracy: 0.7013
Epoch 2/10
901/901 [==============================] - 13s 14ms/step - loss: 0.7981 - accuracy: 0.7329 - val_loss: 0.7480 - val_accuracy: 0.7676
Epoch 3/10
901/901 [==============================] - 13s 15ms/step - loss: 0.7253 - accuracy: 0.7645 - val_loss: 0.7299 - val_accuracy: 0.7708
Epoch 4/10
901/901 [==============================] - 12s 13ms/step - loss: 0.6880 - accuracy: 0.7758 - val_loss: 0.6616 - val_accuracy: 0.7743
Epoch 5/10
901/901 [==============================] - 12s 13ms/step - loss: 0.6574 - accuracy: 0.7778 - val_loss: 0.6406 - val_accuracy: 0.7819
Epoch 6/10
901/901 [==============================] - 12s 13ms/step - loss: 0.6436 - accuracy: 0.7797 - val_loss: 0.6244 - val_accuracy: 0.7831
Epoch 7/10
901/901 [==============================] - 12s 13ms/step - loss: 0.6348 - accuracy: 0.7803 - val_loss: 0.6195 - val_accuracy:

In [14]:
import pickle

# Save the Model
stacked_model.save('hybrid_aclr_model.h5')

# Save the Scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

# Save the Label Encoders
for col, le in label_encoders.items():
    with open(f'{col}_encoder.pkl', 'wb') as encoder_file:
        pickle.dump(le, encoder_file)

# Save the Attack Category Encoder
with open('attack_cat_encoder.pkl', 'wb') as attack_cat_file:
    pickle.dump(attack_cat_encoder, attack_cat_file)

print("Model and preprocessing objects saved successfully!")


Model and preprocessing objects saved successfully!


C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from tensorflow.keras.models import load_model

# Function to Load the Model and Preprocessing Objects
def load_trained_objects():
    # Load the trained hybrid model
    model = load_model('hybrid_aclr_model.h5')
    
    # Load the scaler
    with open('scaler.pkl', 'rb') as scaler_file:
        scaler = pickle.load(scaler_file)
    
    # Load the label encoders
    encoders = {}
    for col in ['proto', 'service']:
        with open(f'{col}_encoder.pkl', 'rb') as encoder_file:
            encoders[col] = pickle.load(encoder_file)
    
    # Load the attack category encoder
    with open('attack_cat_encoder.pkl', 'rb') as attack_cat_file:
        attack_cat_encoder = pickle.load(attack_cat_file)

    return model, scaler, encoders, attack_cat_encoder

# Function to Preprocess User Input
def preprocess_user_input(user_input, encoders, scaler):
    # Encode categorical features
    for col in ['proto', 'service']:
        user_input[col] = encoders[col].transform([user_input[col]])[0]

    # Scale numerical features
    input_scaled = scaler.transform(pd.DataFrame([user_input]))

    # Reshape for model input
    input_reshaped = np.expand_dims(input_scaled, axis=-1)

    return input_scaled, input_reshaped

# Function to Predict Attack Type from User Input
def predict_attack_type_from_user_input(user_input):
    # Load the trained objects
    model, scaler, encoders, attack_cat_encoder = load_trained_objects()

    # Preprocess the user input
    input_scaled, input_reshaped = preprocess_user_input(user_input, encoders, scaler)

    # Predict the attack category
    prediction = model.predict([input_scaled, input_reshaped, input_reshaped, input_reshaped])
    attack_category = attack_cat_encoder.inverse_transform([np.argmax(prediction, axis=1)])[0]

    return attack_category

# Example Manual Input
manual_test = {
    'proto': 'tcp',
    'service': 'http',
    'sbytes': 4000,
    'dbytes': 3000,
    'sttl': 64,
    'swin': 1024,
    'dwin': 512,
    'rate': 0.5,
    'smean': 50,
    'dmean': 40,
}

# Predict the Attack Type
predicted_attack = predict_attack_type_from_user_input(manual_test)
print(f"Predicted Botnet Attack Type: {predicted_attack}")


1/1 [==============================] - 1s 576ms/step
Predicted Botnet Attack Type: Exploits


C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
